## MIST experiments

In [1]:
import numpy as np
import pandas as pd
import os
import glob
from astropy.io import fits
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import trange
from astropy.time import Time
import lightkurve as lk
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_context('paper')

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
from isochrones.mist import MISTIsochroneGrid

grid = MISTIsochroneGrid()
print(len(grid.df))
grid.df.head()  # Just the first few rows

Holoviews not imported. Some visualizations will not be available.
PyMultiNest not imported.  MultiNest fits will not work.
/Users/mag3842/miniforge3/envs/disperse_env/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/mag3842/miniforge3/envs/disperse_env/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


1494453


eep  age       feh      mass  initial_mass  \
log10_isochrone_age_yr feh  EEP                                               
5.0                    -4.0 35    35  5.0 -3.978406  0.100000      0.100000   
                            36    36  5.0 -3.978406  0.102885      0.102885   
                            37    37  5.0 -3.978406  0.107147      0.107147   
                            38    38  5.0 -3.978406  0.111379      0.111379   
                            39    39  5.0 -3.978406  0.115581      0.115581   

                                   radius   density   logTeff         Teff  \
log10_isochrone_age_yr feh  EEP                                              
5.0                    -4.0 35   1.106082  0.104181  3.617011  4140.105252   
                            36   1.122675  0.102504  3.618039  4149.909661   
                            37   1.147702  0.099918  3.619556  4164.436984   
                            38   1.173015  0.097284  3.621062  4178.903372   
                            39   1.198615  0.094623  3.622555  4193.289262   

                                     logg      logL      Mbol   delta_nu  \
log10_isochrone_age_yr feh  EEP                                            
5.0                    -4.0 35   3.350571 -0.489734  5.964335  37.987066   
                            36   3.347798 -0.472691  5.921728  37.739176   
                            37   3.343658 -0.447471  5.858678  37.345115   
                            38   3.339612 -0.422498  5.796244  36.923615   
                            39   3.335660 -0.397776  5.734440  36.473151   

                                     nu_max  phase   dm_deep  
log10_isochrone_age_yr feh  EEP                               
5.0                    -4.0 35   299.346079   -1.0  0.002885  
                            36   298.570836   -1.0  0.003573  
                            37   297.180748   -1.0  0.004247  
                            38   295.526946   -1.0  0.004217  
                            39   293.589960   -1.0  0.004189

In [3]:
grid.df

eep   age       feh      mass  \
log10_isochrone_age_yr feh  EEP                                    
5.0                    -4.0 35      35   5.0 -3.978406  0.100000   
                            36      36   5.0 -3.978406  0.102885   
                            37      37   5.0 -3.978406  0.107147   
                            38      38   5.0 -3.978406  0.111379   
                            39      39   5.0 -3.978406  0.115581   
...                                ...   ...       ...       ...   
10.3                    0.5 1706  1706  10.3  0.585881  0.532726   
                            1707  1707  10.3  0.585884  0.532730   
                            1708  1708  10.3  0.585889  0.532735   
                            1709  1709  10.3  0.585894  0.532741   
                            1710  1710  10.3  0.585901  0.532749   

                                  initial_mass    radius        density  \
log10_isochrone_age_yr feh  EEP                                           
5.0                    -4.0 35        0.100000  1.106082       0.104181   
                            36        0.102885  1.122675       0.102504   
                            37        0.107147  1.147702       0.099918   
                            38        0.111379  1.173015       0.097284   
                            39        0.115581  1.198615       0.094623   
...                                        ...       ...            ...   
10.3                    0.5 1706      0.925792  0.015622  196978.833806   
                            1707      0.925825  0.015513  201160.278202   
                            1708      0.925866  0.015409  205292.361364   
                            1709      0.925919  0.015308  209382.722230   
                            1710      0.925985  0.015211  213420.719611   

                                   logTeff          Teff      logg      logL  \
log10_isochrone_age_yr feh  EEP                                                
5.0                    -4.0 35    3.617011   4140.105252  3.350571 -0.489734   
                            36    3.618039   4149.909661  3.347798 -0.472691   
                            37    3.619556   4164.436984  3.343658 -0.447471   
                            38    3.621062   4178.903372  3.339612 -0.422498   
                            39    3.622555   4193.289262  3.335660 -0.397776   
...                                    ...           ...       ...       ...   
10.3                    0.5 1706  4.402490  25263.303199  7.777159 -1.047903   
                            1707  4.387132  24385.524423  7.783242 -1.115415   
                            1708  4.371789  23539.053774  7.789130 -1.182672   
                            1709  4.356480  22723.735898  7.794844 -1.249617   
                            1710  4.341226  21939.480699  7.800376 -1.316157   

                                      Mbol      delta_nu        nu_max  phase  \
log10_isochrone_age_yr feh  EEP                                                 
5.0                    -4.0 35    5.964335     37.987066  2.993461e+02   -1.0   
                            36    5.921728     37.739176  2.985708e+02   -1.0   
                            37    5.858678     37.345115  2.971807e+02   -1.0   
                            38    5.796244     36.923615  2.955269e+02   -1.0   
                            39    5.734440     36.473151  2.935900e+02   -1.0   
...                                    ...           ...           ...    ...   
10.3                    0.5 1706  7.359758  43738.196186  3.238364e+06    6.0   
                            1707  7.528537  44261.161247  3.341343e+06    6.0   
                            1708  7.696679  44778.694057  3.446438e+06    6.0   
                            1709  7.864044  45293.289878  3.553713e+06    6.0   
                            1710  8.030393  45805.551145  3.662968e+06    6.0   

                                   dm_deep  
log10_isochrone_age_yr feh  EEP   

In [8]:
! head -n 20 ../../data/MIST/MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.4_EEPS/00098M.track.eep

# MIST version number  = 1.2     
# MESA revision number =     7503
# --------------------------------------------------------------------------------------
#  Yinit        Zinit   [Fe/H]   [a/Fe]  v/vcrit                                        
# 0.2703  1.42857E-02     0.00     0.00     0.40
# --------------------------------------------------------------------------------------
#     initial_mass   N_pts   N_EEP   N_col   phase        type
# 9.8000000000E-01    1710      10      77     YES    low-mass
# EEPs:        1     202     353     454     605     631     707     808    1409    1710
# --------------------------------------------------------------------------------------
#                               1                               2                               3                               4                               5                               6                               7                               8                               9                       

In [12]:
#! head -n 20 ../../data/MIST/MIST_v1.2_vvcrit0.4_full_isos/MIST_v1.2_feh_m1.50_afe_p0.0_vvcrit0.4_full.iso